In [1]:
import pandas as pd

# 既存のデータエクセルを再整理

## やったこと

- 既存のデータエクセルをデータベースのテーブルとして正規化を行った。

## 既存のデータエクセルの読み込み

In [2]:
ex_table_rn = pd.read_csv('ROOM_NAME.csv')

In [3]:
ex_table_rn.head(n=3)

,検索キー特定用識別子,検索キー,建物用途コード,室分類コード,建物用途名称,室用途名称,室用途名略称１,室用途名略称２,室用途名略称３
0,事務所等事務室,O-1,Office,OfficeRoom,事務所等,事務室,NaN,NaN,NaN
1,事務所等電子計算機器事務室,O-2,Office,ComputerRoom,事務所等,電子計算機器事務室,電算事務室,NaN,NaN
2,事務所等会議室,O-3,Office,MeetingRoom,事務所等,会議室,NaN,NaN,NaN


In [4]:
ex_table_rs = pd.read_csv('ROOM_SPEC.csv')

In [5]:
ex_table_rs.head(n=3)

,検索キー,建物用途コード,室分類コード,建物用途名称,室用途名称,室用途名略称,カレンダーパターンコード,WSCパターン,照明発熱参照値,人体発熱参照値,...,基準設定湯使用量.1,年間給湯 日数,(旧)検索キー,(旧)室分類コード,旧室用途名称,空調運転パターン,基準設定湯使用量（洗面）,基準設定湯使用量（シャワー）,基準設定湯使用量（厨房）,基準設定湯使用量（その他）
0,O-1,Office,OfficeRoom,事務所等,事務室,NaN,A,WSC1,12.0,0.10,...,[L/人日],241.0,O-1,1.0,事務室,昼,3.8,0,0,0
1,O-2,Office,ComputerRoom,事務所等,電子計算機器事務室,電算事務室,A,WSC1,12.0,0.10,...,[L/人日],241.0,O-2,2.0,電算事務室,昼,3.8,0,0,0
2,O-3,Office,MeetingRoom,事務所等,会議室,NaN,A,WSC1,10.0,0.25,...,[L/人日],241.0,O-3,3.0,会議室,昼,3.8,0,0,0


In [6]:
ex_table_rc = pd.read_csv('ROOM_COND.csv')

In [7]:
ex_table_rc.head(n=3)

,室同時使用率の識別子,検索キー,建物用途コード,室分類コード,負荷種別コード,室使用パターンコード,負荷種別,室使用パターン名称,T0,T1,...,SchedPercent8,SchedTime9,SchedPercent9,SchedTime10,SchedPercent10,SchedTime11,SchedPercent11,SchedTime12,(旧)検索キー,(旧)室分類コード
0,O-1室同時使用率1,O-1,Office,1,1,1,室同時使用率,室使用パターン1,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,O-1,1.0
1,O-1室同時使用率2,O-1,Office,1,1,2,室同時使用率,室使用パターン2,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,O-1,1.0
2,O-1室同時使用率3,O-1,Office,1,1,3,室同時使用率,室使用パターン3,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,O-1,1.0


In [8]:
ex_cdr = pd.read_csv('CALENDAR.csv')

In [9]:
ex_cdr.head(n=3)

,カレンダーパターン検索番号,日付,曜日,パターンA,パターンB,パターンC,パターンD,パターンE,パターンF
0,1,1月1日,木,3,3,3,3,3,3
1,2,1月2日,金,3,3,3,3,3,3
2,3,1月3日,土,3,3,3,3,3,2


## （そこそこ）正規化したテーブルを作成する

In [10]:
pd.read_csv('マスタテーブル名称.csv')

,マスタ名称,英語名,変数名
0,建物用途・室用途マスタ,Building Usage and Room Usage Master,m_BURU
1,カレンダーパターンマスタ,CalendarPatternMaster,m_CP
2,湯使用量マスタ,Hot Water Usage Master,m_HWU
3,室同時使用率マスタ,Room Simultaneously Use Rate Master,m_RSUR
4,照明発熱密度比率マスタ,Lighting Heat Generation Density Rate Master,m_LHGDR
5,人体発熱密度比率マスタ,Human Heat Generation Density Rate Master,m_HHGDR
6,機器発熱密度比率マスタ,Appliance Heat Generation Density Rate Master,m_AHGDR


レコードを表示する関数

In [11]:
def get_record(master_name: str):
    """
    指定したマスタ名称のラベル・ユニーク・説明を返す関数
    Args:
        master_name: マスタ名称
    """
    mtr = pd.read_csv('マスタテーブルレコード.csv')
    return mtr[mtr['テーブル名']==master_name].filter(items=["ラベル", "ユニーク", "説明"]).style.hide_index()

In [12]:
def are_all_data_not_null(d: pd.DataFrame):
    """
    すべての値がNull値ではないことを確認する
    Args:
        確認する pandas DataFrame
    Returns:
        OK または NG の文字列
    """
    if d.isnull().all().values.any():
        return 'NG'
    else:
        return 'OK'

### 建物用途・室用途マスタ

In [13]:
get_record(master_name='建物用途・室用途マスタ')

ラベル,ユニーク,説明
UsageKey,Yes,建物・室の用途を表すキー
BuildingName,No,建物用途
RoomName,No,室用途
RoomNameShort1,No,室用途の略称1
RoomNameShort2,No,室用途の略称2
RoomNameShort3,No,室用途の略称3


In [14]:
m_BURU = ex_table_rn.filter(
    items=["検索キー", "建物用途名称", "室用途名称", "室用途名略称１", "室用途名略称２", "室用途名略称３"]
).set_axis(
    labels=['UsageKey','BuildingName','RoomName','RoomNameShort1','RoomNameShort2','RoomNameShort3'], axis=1
).astype(str).set_index(keys='UsageKey', verify_integrity=True)
are_all_data_not_null(d=m_BURU)

'OK'

In [15]:
m_BURU.head()

,BuildingName,RoomName,RoomNameShort1,RoomNameShort2,RoomNameShort3
UsageKey,,,,,
O-1,事務所等,事務室,nan,nan,nan
O-2,事務所等,電子計算機器事務室,電算事務室,nan,nan
O-3,事務所等,会議室,nan,nan,nan
O-4,事務所等,喫茶室,nan,nan,nan
O-5,事務所等,社員食堂,nan,nan,nan


BuildingName と RoomName の組み合わせに重複が無いことを確認

In [16]:
m_BURU.duplicated(subset=['BuildingName', 'RoomName']).value_counts()

False    210
dtype: int64

### カレンダーパターンマスタ

In [17]:
get_record(master_name='カレンダーパターンマスタ')

ラベル,ユニーク,説明
UsageKey,Yes,建物・室の用途を表すキー
CalendarPattern,No,カレンダーパターン


In [18]:
m_CP =ex_table_rs.filter(
    items=["検索キー", "カレンダーパターンコード"]
).set_axis(
    labels=['UsageKey','CalendarPattern'], axis=1
).astype(str).set_index(keys='UsageKey', verify_integrity=True)
are_all_data_not_null(d=m_CP)

'OK'

In [19]:
m_CP.head()

,CalendarPattern
UsageKey,
O-1,A
O-2,A
O-3,A
O-4,A
O-5,A


CalendarPattern 列に値 A, B, C, D, E, F 以外の値が入っていないことの確認

In [20]:
m_CP['CalendarPattern'].value_counts()

A    161
D     22
E     12
F     10
B      4
C      2
Name: CalendarPattern, dtype: int64

### 湯使用量マスタ

In [21]:
get_record(master_name='湯使用量マスタ')

ラベル,ユニーク,説明
UsageKey,Yes,建物・室の用途を表すキー
Density,No,人体発熱参照値
Standard,No,基準設定湯使用量
Unit,No,単位
WashRoom,No,基準設定湯使用量（洗面）
ShowerRoom,No,基準設定湯使用量（シャワー）
Kitchen,No,基準設定湯使用量（厨房）
Other,No,基準設定湯使用量（その他）


In [22]:
m_HWU = ex_table_rs[ex_table_rs["年間給湯　日数"]>0].filter(
    items=["検索キー", "人体発熱参照値", "基準設定湯使用量", "基準設定湯使用量.1", "基準設定湯使用量（洗面）", "基準設定湯使用量（シャワー）", "基準設定湯使用量（厨房）", "基準設定湯使用量（その他）"]
).set_axis(
    labels=['UsageKey', 'Density', 'Standard', 'Unit', 'WashRoom', 'ShowerRoom', 'Kitchen', 'Other'], axis=1
).astype(
    {
        'UsageKey': str,
        'Density': float,
        'Standard': float,
        'Unit': str,
        'WashRoom': float,
        'ShowerRoom': float,
        'Kitchen': float,
        'Other': float
    }
).set_index(keys='UsageKey', verify_integrity=True).replace(
    {'Unit': {'[L/人日]':'L/人日', '[L/m2日]':'L/m2日', '[L/床日]':'L/床日'}}
)
are_all_data_not_null(d=m_HWU)

'OK'

In [23]:
m_HWU.head()

,Density,Standard,Unit,WashRoom,ShowerRoom,Kitchen,Other
UsageKey,,,,,,,
O-1,0.10,3.8,L/人日,3.8,0.0,0.0,0.0
O-2,0.10,3.8,L/人日,3.8,0.0,0.0,0.0
O-3,0.25,3.8,L/人日,3.8,0.0,0.0,0.0
O-4,0.25,32.0,L/m2日,0.0,0.0,32.0,0.0
O-5,0.50,48.0,L/m2日,0.0,0.0,48.0,0.0


Unit が、「L/人日」, 「L/m2日」, 「L/床日」 以外の値をとっていないことの確認

In [24]:
m_HWU['Unit'].value_counts()

L/人日     79
L/m2日    18
L/床日      2
Name: Unit, dtype: int64

### 室同時使用率マスタ

In [25]:
get_record(master_name='室同時使用率マスタ')

ラベル,ユニーク,説明
UsageKey,Yes,建物・室の用途を表すキー
Pattern,No,室使用パターン名称
0,No,時刻0
1,No,時刻1
2,No,時刻2
3,No,時刻3
4,No,時刻4
5,No,時刻5
6,No,時刻6
7,No,時刻7


In [26]:
m_RSUR = ex_table_rc[ex_table_rc['負荷種別']=='室同時使用率'].replace(
    {'室使用パターン名称':{'室使用パターン1': 1, '室使用パターン2': 2, '室使用パターン3': 3}}
).filter(
    items=['検索キー', '室使用パターン名称', 'T0', 'T1', 'T2', 'T3', 'T4', 'T5', 'T6', 'T7', 'T8', 'T9', 'T10', 'T11', 'T12', 'T13', 'T14', 'T15', 'T16', 'T17', 'T18', 'T19', 'T20', 'T21', 'T22', 'T23']
).set_axis(
    labels=['UsageKey', 'Pattern', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23'], axis=1
).astype(
    {
        'UsageKey': str,
        'Pattern': int,
        '0': float,
        '1': float,
        '2': float,
        '3': float,
        '4': float,
        '5': float,
        '6': float,
        '7': float,
        '8': float,
        '9': float,
        '10': float,
        '11': float,
        '12': float,
        '13': float,
        '14': float,
        '15': float,
        '16': float,
        '17': float,
        '18': float,
        '19': float,
        '20': float,
        '21': float,
        '22': float,
        '23': float
    }
).set_index(keys=['UsageKey', 'Pattern'], verify_integrity=True)
are_all_data_not_null(d=m_RSUR)

'OK'

In [27]:
m_RSUR.head(n=9)

0    1    2    3    4    5    6    7    8    9  ...   14  \
UsageKey Pattern                                                    ...        
O-1      1        0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  1.0  1.0  ...  1.0   
         2        0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0   
         3        0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0   
O-2      1        0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  1.0  1.0  ...  1.0   
         2        0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0   
         3        0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0   
O-3      1        0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.5  0.5  ...  0.5   
         2        0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0   
         3        0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0   

                   15   16   17   18   19   20   21   22   23  
UsageKey Pattern                                               
O-1      1        1.0  1.0  1.0  1.0  1.0  1.0  0.0  0.0  0.0  
         2        0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
         3        0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
O-2      1        1.0  1.0  1.0  1.0  1.0  1.0  0.0  0.0  0.0  
         2        0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
         3        0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
O-3      1        0.5  0.5  0.5  0.0  0.0  0.0  0.0  0.0  0.0  
         2        0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
         3        0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  

[9 rows x 24 columns]

Pattern に 1,2,3 以外の値が入っていないことの確認

In [28]:
(m_RSUR.reset_index())['Pattern'].value_counts()

3    147
2    147
1    147
Name: Pattern, dtype: int64

### 照明発熱密度比率マスタ

In [29]:
get_record(master_name='照明発熱密度比率マスタ')

ラベル,ユニーク,説明
Pattern,No,室使用パターン名称
0,No,時刻0
1,No,時刻1
2,No,時刻2
3,No,時刻3
4,No,時刻4
5,No,時刻5
6,No,時刻6
7,No,時刻7
8,No,時刻8


In [30]:
m_LHGDR = ex_table_rc[ex_table_rc['負荷種別']=='照明発熱密度比率'].filter(
    items=['検索キー', '室使用パターンコード', 'T0', 'T1', 'T2', 'T3', 'T4', 'T5', 'T6', 'T7', 'T8', 'T9', 'T10', 'T11', 'T12', 'T13', 'T14', 'T15', 'T16', 'T17', 'T18', 'T19', 'T20', 'T21', 'T22', 'T23']
).set_axis(
    labels=['UsageKey', 'Pattern', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23'], axis=1
).astype(
    {
        'UsageKey': str,
        'Pattern': int,
        '0': float,
        '1': float,
        '2': float,
        '3': float,
        '4': float,
        '5': float,
        '6': float,
        '7': float,
        '8': float,
        '9': float,
        '10': float,
        '11': float,
        '12': float,
        '13': float,
        '14': float,
        '15': float,
        '16': float,
        '17': float,
        '18': float,
        '19': float,
        '20': float,
        '21': float,
        '22': float,
        '23': float
    }
).set_index(keys=['UsageKey', 'Pattern'], verify_integrity=True)
are_all_data_not_null(d=m_LHGDR)

'OK'

In [31]:
m_LHGDR.head(n=9)

0    1    2    3    4    5    6    7    8    9  ...   14  \
UsageKey Pattern                                                    ...        
O-1      1        0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  1.0  ...  1.0   
         2        0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0   
         3        0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0   
O-2      1        0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  1.0  ...  1.0   
         2        0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0   
         3        0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0   
O-3      1        0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  ...  1.0   
         2        0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0   
         3        0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0   

                   15   16   17   18   19   20   21   22   23  
UsageKey Pattern                                               
O-1      1        1.0  1.0  1.0  1.0  1.0  0.8  0.0  0.0  0.0  
         2        0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
         3        0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
O-2      1        1.0  1.0  1.0  1.0  1.0  0.8  0.0  0.0  0.0  
         2        0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
         3        0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
O-3      1        1.0  1.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  
         2        0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
         3        0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  

[9 rows x 24 columns]

Pattern に 1,2,3 以外の値が入っていないことの確認

In [32]:
(m_LHGDR.reset_index())['Pattern'].value_counts()

3    147
2    147
1    147
Name: Pattern, dtype: int64

### 人体発熱密度比率マスタ

In [33]:
get_record(master_name='人体発熱密度比率マスタ')

ラベル,ユニーク,説明
Pattern,No,室使用パターン名称
0,No,時刻0
1,No,時刻1
2,No,時刻2
3,No,時刻3
4,No,時刻4
5,No,時刻5
6,No,時刻6
7,No,時刻7
8,No,時刻8


In [34]:
m_HHGDR = ex_table_rc[ex_table_rc['負荷種別']=='人体発熱密度比率'].filter(
    items=['検索キー', '室使用パターンコード', 'T0', 'T1', 'T2', 'T3', 'T4', 'T5', 'T6', 'T7', 'T8', 'T9', 'T10', 'T11', 'T12', 'T13', 'T14', 'T15', 'T16', 'T17', 'T18', 'T19', 'T20', 'T21', 'T22', 'T23']
).set_axis(
    labels=['UsageKey', 'Pattern', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23'], axis=1
).astype(
    {
        'UsageKey': str,
        'Pattern': int,
        '0': float,
        '1': float,
        '2': float,
        '3': float,
        '4': float,
        '5': float,
        '6': float,
        '7': float,
        '8': float,
        '9': float,
        '10': float,
        '11': float,
        '12': float,
        '13': float,
        '14': float,
        '15': float,
        '16': float,
        '17': float,
        '18': float,
        '19': float,
        '20': float,
        '21': float,
        '22': float,
        '23': float
    }
).set_index(keys=['UsageKey', 'Pattern'], verify_integrity=True)
are_all_data_not_null(d=m_HHGDR)

'OK'

In [35]:
m_HHGDR.head(n=9)

0    1    2    3    4    5    6    7    8    9  ...   14  \
UsageKey Pattern                                                    ...        
O-1      1        0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  1.0  ...  1.0   
         2        0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0   
         3        0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0   
O-2      1        0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  1.0  ...  1.0   
         2        0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0   
         3        0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0   
O-3      1        0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  ...  1.0   
         2        0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0   
         3        0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0   

                   15   16   17   18   19   20   21   22   23  
UsageKey Pattern                                               
O-1      1        1.0  1.0  1.0  0.5  0.3  0.2  0.0  0.0  0.0  
         2        0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
         3        0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
O-2      1        1.0  1.0  1.0  0.5  0.3  0.2  0.0  0.0  0.0  
         2        0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
         3        0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
O-3      1        1.0  1.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  
         2        0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
         3        0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  

[9 rows x 24 columns]

Pattern に 1,2,3 以外の値が入っていないことの確認

In [36]:
(m_HHGDR.reset_index())['Pattern'].value_counts()

3    147
2    147
1    147
Name: Pattern, dtype: int64

### 機器発熱密度比率マスタ

In [37]:
get_record(master_name='機器発熱密度比率マスタ')

ラベル,ユニーク,説明
Pattern,No,室使用パターン名称
0,No,時刻0
1,No,時刻1
2,No,時刻2
3,No,時刻3
4,No,時刻4
5,No,時刻5
6,No,時刻6
7,No,時刻7
8,No,時刻8


In [38]:
m_AHGDR = ex_table_rc[ex_table_rc['負荷種別']=='機器発熱密度比率'].filter(
    items=['検索キー', '室使用パターンコード', 'T0', 'T1', 'T2', 'T3', 'T4', 'T5', 'T6', 'T7', 'T8', 'T9', 'T10', 'T11', 'T12', 'T13', 'T14', 'T15', 'T16', 'T17', 'T18', 'T19', 'T20', 'T21', 'T22', 'T23']
).set_axis(
    labels=['UsageKey', 'Pattern', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23'], axis=1
).astype(
    {
        'UsageKey': str,
        'Pattern': int,
        '0': float,
        '1': float,
        '2': float,
        '3': float,
        '4': float,
        '5': float,
        '6': float,
        '7': float,
        '8': float,
        '9': float,
        '10': float,
        '11': float,
        '12': float,
        '13': float,
        '14': float,
        '15': float,
        '16': float,
        '17': float,
        '18': float,
        '19': float,
        '20': float,
        '21': float,
        '22': float,
        '23': float
    }
).set_index(keys=['UsageKey', 'Pattern'], verify_integrity=True)
are_all_data_not_null(d=m_AHGDR)

'OK'

In [39]:
m_AHGDR.head(n=9)

0     1     2     3     4     5     6     7     8     9  \
UsageKey Pattern                                                               
O-1      1        0.25  0.25  0.25  0.25  0.25  0.25  0.25  0.25  1.00  1.00   
         2        0.25  0.25  0.25  0.25  0.25  0.25  0.25  0.25  0.25  0.25   
         3        0.25  0.25  0.25  0.25  0.25  0.25  0.25  0.25  0.25  0.25   
O-2      1        0.25  0.25  0.25  0.25  0.25  0.25  0.25  0.25  1.00  1.00   
         2        0.25  0.25  0.25  0.25  0.25  0.25  0.25  0.25  0.25  0.25   
         3        0.25  0.25  0.25  0.25  0.25  0.25  0.25  0.25  0.25  0.25   
O-3      1        0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  1.00   
         2        0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00   
         3        0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00   

                  ...    14    15    16    17    18    19    20    21    22  \
UsageKey Pattern  ...                                                         
O-1      1        ...  1.00  1.00  1.00  1.00  1.00  0.50  0.50  0.25  0.25   
         2        ...  0.25  0.25  0.25  0.25  0.25  0.25  0.25  0.25  0.25   
         3        ...  0.25  0.25  0.25  0.25  0.25  0.25  0.25  0.25  0.25   
O-2      1        ...  1.00  1.00  1.00  1.00  1.00  0.50  0.50  0.25  0.25   
         2        ...  0.25  0.25  0.25  0.25  0.25  0.25  0.25  0.25  0.25   
         3        ...  0.25  0.25  0.25  0.25  0.25  0.25  0.25  0.25  0.25   
O-3      1        ...  1.00  1.00  1.00  1.00  0.00  0.00  0.00  0.00  0.00   
         2        ...  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00   
         3        ...  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00   

                    23  
UsageKey Pattern        
O-1      1        0.25  
         2        0.25  
         3        0.25  
O-2      1        0.25  
         2        0.25  
         3        0.25  
O-3      1        0.00  
         2        0.00  
         3        0.00  

[9 rows x 24 columns]

Pattern に 1,2,3 以外の値が入っていないことの確認

In [40]:
(m_AHGDR.reset_index())['Pattern'].value_counts()

3    147
2    147
1    147
Name: Pattern, dtype: int64

### カレンダーマスタ

In [41]:
get_record(master_name='カレンダーマスタ')

ラベル,ユニーク,説明


In [42]:
m_CDR = ex_cdr.filter(
    items=['カレンダーパターン検索番号', '日付', '曜日', 'パターンA', 'パターンB', 'パターンC', 'パターンD', 'パターンE', 'パターンF']
).set_axis(
    labels=['CalendarPatternKey', 'Date', 'DayOfTheWeek', 'A', 'B', 'C', 'D', 'E', 'F'], axis=1
).astype(
    {
        'CalendarPatternKey': int,
        'Date': str,
        'DayOfTheWeek': str,
        'A': int,
        'B': int,
        'C': int,
        'D': int,
        'E': int,
        'F': int
    }
).set_index(keys=['CalendarPatternKey'], verify_integrity=True)
are_all_data_not_null(d=m_CDR)

'OK'

In [43]:
m_CDR.head()

,Date,DayOfTheWeek,A,B,C,D,E,F
CalendarPatternKey,,,,,,,,
1,1月1日,木,3,3,3,3,3,3
2,1月2日,金,3,3,3,3,3,3
3,1月3日,土,3,3,3,3,3,2
4,1月4日,日,3,2,2,2,2,2
5,1月5日,月,1,2,2,3,3,1


In [44]:
m_CDR['A'].value_counts()

1    241
3     73
2     51
Name: A, dtype: int64

In [45]:
m_CDR['B'].value_counts()

1    196
2    163
3      6
Name: B, dtype: int64

In [46]:
m_CDR['C'].value_counts()

2    196
1    163
3      6
Name: C, dtype: int64

In [47]:
m_CDR['D'].value_counts()

1    229
2    118
3     18
Name: D, dtype: int64

In [48]:
m_CDR['E'].value_counts()

1    189
2    118
3     58
Name: E, dtype: int64

In [49]:
m_CDR['F'].value_counts()

1    132
2    119
3    114
Name: F, dtype: int64

## データの書き出し

In [50]:
for df, file_name in [
    (m_BURU, 'm_BURU'),
    (m_CP, 'm_CP'),
    (m_HWU, 'm_HWU'),
    (m_RSUR, 'm_RSUR'),
    (m_LHGDR, 'm_LHGDR'),
    (m_HHGDR, 'm_HHGDR'),
    (m_AHGDR, 'm_AHGDR'),
    (m_CDR, 'm_CDR')
]:
    df.to_csv(file_name + '.csv', encoding='cp932')